In [ ]:
from dateutil.parser import parse
from dateutil.parser import ParserError
from dateutil.tz import gettz
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import traceback

In [ ]:
# # data format ANSI
# f = r"C:\LinkToBanking.csv"

# date_columns = ['Date','EntryDate', 'ValueDate', 'TransactionDate', 'AnalysisDate']

# # mbcs = ANSI encoding
# df = pd.read_csv(f,sep=";",encoding="mbcs",decimal=",",
#                  delimiter=";",parse_dates=date_columns,
#                  thousands='.')

In [ ]:
f = r"<C:\link to banking file.csv>"
# only read account information

# treat date columns
date_columns = ["Buchungsdatum","Wertstellung"]

df = pd.read_csv(f,sep=";",decimal=",",
                 delimiter=";",nrows=3,
                 thousands='.')
# for this csv we find the account info here
account_info = df.iloc[1][1];

# read csv
df = pd.read_csv(f,sep=";",decimal=",",parse_dates=date_columns,
                 delimiter=";",nrows=999999,skiprows=6,
                 thousands='.')
df["Konto"] = account_info
# convert date
df['JJJJ'] = df['Buchungsdatum'].map(lambda x: x.strftime('%Y'))
df['JJJJMM'] = df['Buchungsdatum'].map(lambda x: x.strftime('%Y%m'))
df['JJJJWW'] = df['Buchungsdatum'].map(lambda x: x.strftime('%Y%U'))

In [ ]:
# drop columns we do not need
drop_cols = ['UCI','Mandat ID', 'Abweichender Debitor', 'Abweichender Kreditor','Referenznummer', 'Wertstellung']
df.drop(labels=drop_cols, axis=1,  inplace=True)

In [ ]:
# look for keyword categories in booking description
spending_vocabulary = ["store","burger"]
#drop cols from previous run
df.drop(labels=spending_vocabulary, axis=1, inplace=True)
#data col containing description
data = df.Umsatzinformation

# transform keywords / add it to dataframe  into columns using CountVerctorizer
vectorizer = CountVectorizer(lowercase=True,vocabulary=spending_vocabulary,ngram_range=(1,2))
X = vectorizer.fit_transform(data)

df_category =  pd.DataFrame(data=X.toarray(),columns=vectorizer.get_feature_names())
df_category
df = pd.concat([df, df_category], axis=1)

# get new category
df["Essen"] = df[spending_vocabulary].max(axis=1)
df


In [ ]:
# transfer separate balance cols (debit,credit) into one sum column
def get_balance(debit,credit):
    balance = 0
    if not(np.isnan(debit)):
        balance += -1. * debit
    if not(np.isnan(credit)):
        balance += credit
    return balance

balance_cols = ['Soll (EUR)','Haben (EUR)']
df["Betrag"] = df[balance_cols].apply(lambda f:get_balance(f["Soll (EUR)"],f["Haben (EUR)"]),axis=1)
df

In [ ]:
# aggregate by year month, category
df_agg = df.groupby(by=["JJJJ","JJJJMM","Essen"]).sum()
df_agg

In [ ]:
df_agg[df_agg.index.get_level_values('Essen') == 1]